In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
if os.path.exists('../secrets/token.json'):
  creds = Credentials.from_authorized_user_file('../secrets/token.json', SCOPES)

In [ ]:
service = build('gmail', 'v1', credentials=creds)

In [ ]:
results = service.users().messages().list(userId='me', labelIds=['SENT']).execute()
results.get('messages')

In [ ]:
results.get('messages')[0].get('id')

In [ ]:
# Load the actual message
top_message = results.get('messages')[0].get('id')
message = service.users().messages().get(userId="me", id=top_message).execute()
message

In [ ]:
import base64

parts = message.get('payload').get('parts')
part = None
for p in parts:
  if p.get('mimeType') == 'text/plain':
    part = p
    break
    
content = None
if part:
  data = part.get('body').get('data')
  encoded = bytes(str(data),encoding='utf-8')
  content = base64.urlsafe_b64decode(encoded).decode('utf-8')

## Quoted Content
I am trying to clear out the quoted content shit...

In [ ]:
import re

lines = content.split('\n')
message = []
for line in lines:
  result = re.search(r'^On.*< \S+ > wrote:', line)
  if result:
    break
  else:
    message.append(line)

message = '\n'.join(message)
print(message)

# Now, lets get a thread

In [ ]:
top_message = top_message = results.get('messages')[0]
top_message
thread = service.users().threads().get(userId="me", id=top_message.get("threadId")).execute()
thread

In [ ]:
# Trying to get plain text unquoted content for a message
# this way I can loop through a thread and get all the content

# TODO add some kind of error handling
def parse_message (json):
  parts = json.get('payload').get('parts')
  part = None
  # Working right now off plaintext
  for p in parts:
    if p.get('mimeType') == 'text/plain':
      part = p
      break
      
  content = None
  if part:
    data = part.get('body').get('data')
    encoded = bytes(str(data),encoding='utf-8')
    content = base64.urlsafe_b64decode(encoded).decode('utf-8')
    
  lines = content.split('\n')
  message = []
  for line in lines:
    result = re.search(r'^On.*<.*>,? wrote:', line)
    if result:
      break
    else:
      message.append(line)

  message = '\n'.join(message)
  return message
    

In [ ]:
print(parse_message(thread.get('messages')[2]))

In [ ]:
json = service.users().messages().get(userId="me", id=top_message.get('id')).execute()
def get_previous_messages(json):
  thread = service.users().threads().get(userId="me", id=top_message.get("threadId")).execute()
  message_id = json.get('id')

  messages = thread.get('messages')
  previous_messages = []
  for message in messages:
    if message_id != message.get('id'):
      previous_messages.append(parse_message(message))
    else:
      break
      
  return previous_messages

previous_messages = get_previous_messages(json)

In [ ]:
print("THREAD")
for m in previous_messages:
  print(m)
  print("=== \n")
  
print("\nRESPONSE")
print(parse_message(json))
